# Lab 04 - Logistic Regression and ROC

In previous labs we investigated the behavior of the linear regression model as well as of polynomial fitting using linear regression. In the latter we have also seen how the model complexity (captured by the degree $k$ of the fitted polynomial) and the noise levels impact the fitted model and its performance over a given test set.

The following lab extends linear regression into a classification problem using the logistic regression model. Here, attempt to model the *probability* of a given sample $\mathbf{x}$ to be classified as $1$. To do so, we assume that: $$ y|\mathbf{x} \sim Ber\left(\phi_\mathbf{w}\left(\mathbf{x}\right)\right),\quad \phi_\mathbf{w}\left(\mathbf{x}\right) = sigm\left(\mathbf{x}^\top \mathbf{w}\right) $$

Namely, that $y$ has a Bernoulli distribution where the probability that $y=1$ depends on the model $\mathbf{w}$ and the observation $\mathbf{x}$ and is given by $sigm\left(\mathbf{x}^\top \mathbf{w}\right)$. These assumptions yield the following hypothesis class of logistic regression: $$ \mathcal{H}_{logistic}= \left\{\mathbf{x}\to sigm\left( \mathbf{x}^\top\mathbf{w}\right) \,\,\Big| \,\,\mathbf{w}\in\mathbb{R}^{d+1}\right\} $$

Notice that the hypothesis class defined above does not provide us with classifiers, i.e. functions that return either $0$ or $1$ but rather values in $\left[0,1\right]$. To turn the outputted value (which is thought of as the probability of the sample being classified as $1$) into a binary decision we must also specify a threshold value $\alpha$. Then if the value is greater or equals to $\alpha$ we predict the class as being $1$ and otherwise as $0$.

In [11]:
import sys
sys.path.append("../")
from utils import *

np.random.seed(0)
c = [custom[0], custom[-1]]

Let us begin with generating a two-class dataset, with the points of each class generated from some bivariate Gaussian distribution. We visualize the data using a scatter plot as well as class-specific histograms depicting the marginal distributions of samples in each class.

In [12]:
n0, n1 = 1000, 1000
mu = np.array([[0, 0], [4, 5]])
cov = np.array([[[1, .5], [.5, 1]], [[5, 3.5], [3.5, 8]]])


X = np.r_[np.random.multivariate_normal(mu[0], cov[0], n0),
          np.random.multivariate_normal(mu[1], cov[1], n1)]
y = np.r_[[0]*n0, [1]*n1]


go.Figure([
    go.Scatter(x = X[:,0], y=X[:, 1], mode = 'markers', marker = dict(color=y, colorscale=c, reversescale=True, size = 3)),
    go.Histogram(x = X[y==0, 0], histnorm="probability density", yaxis = 'y2', opacity = .5, marker = dict(color = c[1][1])),
    go.Histogram(x = X[y==1, 0], histnorm="probability density", yaxis = 'y2', opacity = .5, marker = dict(color = c[0][1])),
    go.Histogram(y = X[y==0, 1], histnorm="probability density", xaxis = 'x2', opacity = .5, marker = dict(color = c[1][1])),
    go.Histogram(y = X[y==1, 1], histnorm="probability density", xaxis = 'x2', opacity = .5, marker = dict(color = c[0][1]))
    ], layout=go.Layout(
    xaxis =  dict(zeroline = False, domain = [0,0.85], showgrid = False),
    yaxis =  dict(zeroline = False, domain = [0,0.85], showgrid = False),
    xaxis2 = dict(zeroline = False, domain = [0.85,1], showgrid = False),
    yaxis2 = dict(zeroline = False, domain = [0.85,1], showgrid = False),
    hovermode = 'closest', showlegend = False, height=600, width=600,
    title = r"$(1)\text{ Dataset To Fit Logistic Regression Model To}$"
))

We fit a logistic regression model over the data generated above and predict the class assignment probability predicted by the fitted model. Therefore, for each sample we have a value expressing the probability of being assigned as class $1$. In Figure 2 we observe that for most samples the model is very certain of their label since the predicted probability is either close to $0$ or close to $1$. 

Consider what will happen when we transition from a continuous scale to a discrete one? Which samples are most definitly going to be classified as $0$ or $1$? For which samples is the model less certain about their class assignment?

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X, y)
y_prob= model.predict_proba(X)[:, 1]

go.Figure([
    go.Scatter3d(x = X[:, 0], y=X[:, 1], z = [-0.1]*X.shape[0], mode = 'markers',
                 marker = dict(color=y, symbol="circle-open", colorscale=c, reversescale=True, size = 1)),
    go.Scatter3d(x = X[:, 0], y=X[:, 1], z = y_prob, mode = 'markers',
                 marker = dict(color = y_prob, colorscale=custom, reversescale=True, showscale=True, size = 3))],
    layout = go.Layout(title=r"$(2)\text{ Predicted Class Probabilities}$",
                       scene_aspectmode="cube", showlegend=False,
                       scene=dict(xaxis_title = "Feature 1",
                                  yaxis_title = "Feature 2",
                                  zaxis_title = "Probabilty of Assigning Class 1",
                                  camera = dict(eye=dict(x=1, y=-1.8, z=.1)))))


An easier visualization to use for answering the questions above is the histogram below (Figure 3). As before we see that the vast majority of samples are, by the fitted classifier, either of class $0$ or of class $1$, with fewer and fewer samples with probabilities around $0.5$ of being classified as $1$.

Then, we specify a threshold probability over which we classify a sample as being of class $1$. See below how different thresholds influence what samples are above or below the threshold.

In [14]:
fig = px.histogram(x=y_prob, color=y, nbins=50, barmode="overlay",
             color_discrete_sequence=[c[1][1], c[0][1]],
             labels=dict(color=r'$\text{True Class Assignments}$', 
                         x=r'$\text{Probability of Assigning Class }1$'),
             title=r"$(3)\text{ Histogram of Class Assignment Probabilities}$",
             height=350)

frames = [go.Frame(data=go.Scatter(x=[t/10, t/10], y=[0, 600],mode="lines", line=dict(color="black"), showlegend=False), traces=[2]) 
          for t in range(11)]

fig.add_traces(frames[0]["data"][0])\
   .update(frames=frames[1:])\
   .update_layout(updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=1000), 
                                                              AnimationButtons.pause()])])

So by specifying such a threshold we partition the samples into 4 sets (suppose $1$ is the "positive label"):
- True Positives: Samples of class $1$ and that are predicted to be $1$.
- True Negatives: Samples of class $0$ and that are predicted to be $0$.
- False Positives: Samples of class $0$ and that are predicted to be $1$.
- False Negatives: Samples of class $1$ and that are predicted to be $0$.

Confusion matrices are a common way of visualizing these four groups. It is to note that confusion matrices are also used in a wider context where we have two sets of categories $A_1,\ldots,A_k$ and $B_1,\ldots,B_l$ and samples are characterized by both a category of $A$ and $B$. Then, the confusion matrix will show the number of samples in each pair $A_i,B_j,\,\, i\in\left[k\right], j\in\left[l\right]$.

In [15]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=[r"$\text{Assigned Probability}$",r"$\text{Confusion Matrix}$"])

frames = []
for t in range(11):
    # Create histogram of class probabilities and mark with vertical line current threshold
    hist = px.histogram(x=y_prob, color=y, barmode="overlay", nbins=10,
                color_discrete_sequence=[c[1][1], c[0][1]])\
        .add_trace(go.Scatter(x=[t/10, t/10], y=[0, 800],mode="lines", line=dict(color="black"), showlegend=False))
    
    
    # Create a confusion matrix for classifying based on current threshold t/10
    cm = confusion_matrix(y, model.predict_proba(X)[:, 1] >= t/10, labels=[0,1])
    cm = ff.create_annotated_heatmap(cm, y = [r"$y = 0$", r"$y = 1$"], x = [r"$\widehat{y} = 0$", r"$\widehat{y}=1$"],
                                     annotation_text=np.core.defchararray.add(np.array([["TN: ", "FP: "], ["FN: ", "TP: "]]), cm.astype("<U4")),
                                     showscale=True, colorscale="OrRd")

    
    # Create anomation frame using graphs above and their annotations
    for annot in cm.layout.annotations: annot["xref"], annot["yref"] = "x2", "y2"
    frames.append(go.Frame(data=hist.data + cm.data, 
                           layout=go.Layout(annotations=fig.layout.annotations + hist.layout.annotations + cm.layout.annotations),
                           traces=[0,1,2,3]))
    

# Incorporate frames' data and annotations into subplot figures
fig.add_traces(frames[0].data[:3],1,1)\
   .add_traces(frames[0].data[3],1,2)\
   .update(frames=frames)\
   .update_xaxes(title_text=r'$\text{Probability of Assigning Class }1$', row=1, col=1)\
   .update_layout(title=r"$(4)\text{ Confusion Matrix For Different Thresholds}$",
                  height=350, showlegend=False, margin=dict(t=60),
                  annotations = frames[0].layout.annotations, barmode="overlay",
                  updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=1500), AnimationButtons.pause()])])


Based on the confusion matrix (calculated for a given threshold) we are able to define different measures of performance for the classifier. If for example, we want to ask what is the classifier's accuracy we will look at the fraction of correcly classified samples, that is, $Accuracy=TP+TN/(P+N)$.

Similarly, we can define many other useful measures. Two measures that are commonly used in the field of computer science are:
- False Positive Rate: the fraction of false-positives out of all positives: $FPR=FP/N$. This measure describes how unsuccessful the classifier is in specifying negative labels.
- True Positive Rate (sensitivity): the fraction true-positives out of all positives: $TPR=TP/P$. This measure describes how successful the classifier is in specifying positive labels.

These two measures are then combined into a single graph - the ROC curve. In this grahp, each point represents a threshold that yielded a pait of $(FPR,TPR)$. It can help us evaluate the classifiers performance in classifying positive samples and determine the optimal threshold value.

Another use of the ROC curve is to calculate further summary statistics such as the Area Under Curve (AUC). Given randomly chosen positive an negative samples, the AUC equals to the probability that the classifier will output a higher probability of assignment to class $1$ for the positive sample compared to the negative sample.

In [16]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y, y_prob)

go.Figure(
    data=[go.Scatter(x=[0,1], y=[0,1], mode="lines", line=dict(color="black", dash='dash'), name="Random Class Assignment"),
          go.Scatter(x=fpr, y=tpr, mode='markers+lines',text=thresholds, name="", showlegend=False, marker_size=5, marker_color=c[1][1],
                     hovertemplate="<b>Threshold:</b>%{text:.3f}<br>FPR: %{x:.3f}<br>TPR: %{y:.3f}")],
    layout=go.Layout(title=rf"$\text{{ROC Curve Of Fitted Model - AUC}}={auc(fpr, tpr):.6f}$", 
                                 xaxis=dict(title=r"$\text{False Positive Rate (FPR)}$"),
                                 yaxis=dict(title=r"$\text{True Positive Rate (TPR)}$")))

## Time To Think...

In the above we saw how fitting a logistic regression classifier over the given data yields class assignment probabilities which then, by determining a threshold, can be converted into a binday classification. By computing the ROC curve of the fitted model we get a sense into how the model performed in terms of the prediction of positive samples. This can then be summaries further using the AUC statistic.

- Consider the case seen above but this time, what will happen if the samples of the two classes are closer to each other? Modify the generated data above such that the samples of class $1$ (in red) are sampled from: $$\mathcal{N}\left(\left[\begin{array}{c} 2.5\\4\end{array}\right],\left[\begin{array}{cc} 1 & 0.8\\0.8 & 1\end{array}\right]\right)$$ How does this influence the fitted model - is the classifier more or less able to classify the samples correctly? Is the AUC statistic higher or lower than before?
- Consider the case seen above but this time, what will happen if the samples of the two classes are further apart and the variance of samples in the one class is much higher? Modify the generated data above suc hthat the samples of class $1$ are sampled from: $$\mathcal{N}\left(\left[\begin{array}{c} 4\\5\end{array}\right],\left[\begin{array}{cc} 5 & 3.5\\3.5 & 8\end{array}\right]\right)$$ Compared to the setting in the previous bullet, is the classifier more confident about the probabilities it assigns? (in the sense of them being more towards $0$ and $1$). Does the classifier in this case have more misclassifications of the one type compared to the previous bullet?

It is **highly recommented** to further investigate some of the different measures of performance such as [precision and specificity](https://en.wikipedia.org/wiki/Sensitivity_and_specificity). Each different measure captures a different aspect of what was done by the classifier. Different applications tend to care more for different measures. Therefore, depending on the application one might prefer different measures.